In [6]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable

In [7]:
#Old
#WESM_price = np.genfromtxt('210607 RC WESM Forecast.csv',dtype=float,delimiter=',',skip_header=1)

#New
WESM_price = np.genfromtxt('ref/210104 RC WESM Forecast Base Case.csv',dtype=float,delimiter=',',skip_header=1)


These are the modifiable parameters

In [15]:
battery_kwh = 8480
battery_kw = 2500
acinv_efficiency = 0.93 #93% AC Efficiency
dod_allowed = 0.90 # 90% Allowable DOD
start_year = 2021
end_year = 2030

#Assuming a fully charged unit upon delivery, initial charge values can be neglected
initial_charge_kwh = battery_kwh/acinv_efficiency
initial_charge_time = initial_charge_kwh/battery_kw
discharge_kwh = battery_kwh*dod_allowed*acinv_efficiency
discharge_time = discharge_kwh/battery_kw
charge_kwh = discharge_kwh/acinv_efficiency
charge_time = charge_kwh/battery_kw
pcr = float("{:.2f}".format(charge_time%1))
pdr = float("{:.2f}".format(discharge_time%1))
charge_int = np.floor(charge_time)
discharge_int = np.floor(discharge_time)

print("Total Charging Hours:","{:.2f}".format(charge_time))
print("Total Discharging Hours:","{:.2f}".format(discharge_time))

Total Charging Hours: 3.05
Total Discharging Hours: 2.84


This is the main calculating loop.

In [12]:
def calculatePerformanceWESMCharging(year_iter,activity_interval):
    i=0
    WESM_this_year = WESM_price[:,year_iter]
    partial_charge_rate = pcr
    partial_discharge_rate = pdr
    this_year = start_year + year_iter
    charge_time = charge_int
    discharge_time = discharge_int
    charge_hours = int(np.ceil(charge_time+partial_charge_rate))
    discharge_hours = int(np.ceil(discharge_time+partial_discharge_rate))
    bs_charge = 0
    bs_pcharge = 0
    bs_discharge = 0
    bs_pdischarge=0
    charge_cycles = 0
    discharge_cycles = 0
    income = 0
    cost = 0
    print(WESM_this_year)
    battery_state = np.zeros(shape=8760, dtype=int)
    while(i!=len(battery_state)):
        try: 
            charge_window = WESM_this_year[i:i+16]
            print(charge_window)
            cindex = (charge_window).argsort()[:charge_hours]
            print(cindex)
            for j in range(0,len(cindex)):
                this_index = i+cindex[j]
                if j == (len(cindex)-1) and partial_charge_rate>0.0:
                    battery_state[this_index] = 2
                    bs_pcharge = bs_pcharge + 1
                    cost = cost+(WESM_this_year[this_index]*partial_charge_rate*battery_kw)
                else:
                    battery_state[this_index] = 1 # Charging
                    bs_charge = bs_charge+1
                    cost = cost + (WESM_this_year[this_index]*battery_kw)
            charge_cycles = charge_cycles + 1

            print(battery_state[0:24])
            i = i+np.amax(cindex)+1
            print(i)

            discharge_window = WESM_this_year[i:i+activity_interval]
            print(discharge_window)
            dindex = (-discharge_window).argsort()[:discharge_hours]
            print(dindex)
            for j in range(0,len(dindex)):
                this_index = i+dindex[j]
                if j == (len(dindex)-1) and partial_discharge_rate>0.0:
                    battery_state[this_index] = 4
                    bs_pdischarge = bs_pdischarge + 1
                    income = income+(WESM_this_year[this_index]*partial_discharge_rate*battery_kw)
                else:
                    battery_state[this_index] = 3 # Discharging
                    bs_discharge = bs_discharge + 1
                    income = income + (WESM_this_year[this_index]*battery_kw)
            print(battery_state[0:24])
            discharge_cycles = discharge_cycles+1
            i = i+np.amax(dindex)+1
        #print(i)
        except Exception as e:
            #print(i,'//',e)
            break
    count_hours = np.bincount(battery_state)
    hours_charging = bs_charge+ partial_charge_rate*bs_pcharge
    hours_discharging = bs_discharge + partial_discharge_rate*bs_pdischarge
    hours_idle = 8760 - (hours_charging+hours_discharging)
    profit = income - cost
    results = np.array([this_year,(charge_time+partial_charge_rate),(discharge_time+partial_discharge_rate),activity_interval,hours_charging, hours_discharging,hours_idle,charge_cycles,discharge_cycles,round(cost,2),round(income,2),round(profit,2)])
    return results



This is the main loop to generate the results per year.

In [17]:
x = PrettyTable()
x.field_names = ["Year","Charge Time","Discharge Time","Activity Interval","Hours Charging","Hours Discharging","Hours Idle","Charge Cycles","Discharge Cycles","Cost","Income","Profit"]
result = calculatePerformanceWESMCharging(0,16)
x.add_row(result)
print(x)

[1.2092 1.0283 0.9866 ... 3.1809 1.5546 1.4067]
[1.2092 1.0283 0.9866 0.9798 0.9878 1.0005 1.0186 2.5004 2.7076 3.0479
 3.0905 3.0007 2.8261 2.8178 2.9355 2.9806]
[3 2 4 5]
[0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
6
[1.0186 2.5004 2.7076 3.0479 3.0905 3.0007 2.8261 2.8178 2.9355 2.9806
 3.0833 3.6991 3.7575 3.5588 3.3488 3.0878]
[12 11 13]
[0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 3 3 4 0 0 0 0]
[3.3488 3.0878 1.4571 1.3735 1.3248 1.2703 1.1976 1.1334 1.1772 1.2412
 1.2284 2.1553 2.4676 2.7306 2.7677 2.727 ]
[ 7  8  6 10]
[0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 3 3 4 0 0 0 0]
31
[2.1553 2.4676 2.7306 2.7677 2.727  2.6312 2.6132 2.5421 2.5384 2.5874
 3.2819 3.3155 3.0058 2.8765 2.7397 1.3911]
[11 10 12]
[0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 3 3 4 0 0 0 0]
[2.8765 2.7397 1.3911 1.2293 1.3011 1.2105 1.0401 1.0456 1.111  1.1
 1.0795 1.4172 1.6208 2.0151 2.2417 2.193 ]
[ 6  7 10  9]
[0 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 3 3 4 0 0 0 0]
55
[1.4172 1.6208 2.0151 2.2417 2.193  2.1601 2.3284 2.2308 2.173

In [10]:
x = PrettyTable()
x.field_names = ["Year","Charge Time","Discharge Time","Activity Interval","Hours Charging","Hours Discharging","Hours Idle","Charge Cycles","Discharge Cycles","Cost","Income","Profit"]
datatable = np.zeros(shape=(end_year-start_year+1,12))
for i in range(0,(end_year-start_year+1)):
    result = calculatePerformanceWESMCharging(i,16)
    datatable[i] = result
    x.add_row(result)
print(x)
print(datatable)
#datatable.savetxt('results.csv',sep=',',format='%12.5f')
np.savetxt('results.csv',datatable,fmt="%d",delimiter=',')

+--------+-------------+----------------+-------------------+----------------+-------------------+------------+---------------+------------------+-------------+-------------+-------------+
|  Year  | Charge Time | Discharge Time | Activity Interval | Hours Charging | Hours Discharging | Hours Idle | Charge Cycles | Discharge Cycles |     Cost    |    Income   |    Profit   |
+--------+-------------+----------------+-------------------+----------------+-------------------+------------+---------------+------------------+-------------+-------------+-------------+
| 2021.0 |     6.11    |      5.68      |        16.0       |    2232.26     |       2073.2      |  4454.54   |     366.0     |      366.0       | 10704263.82 | 31581970.47 | 20877706.65 |
| 2022.0 |     6.11    |      5.68      |        16.0       |    2232.26     |       2073.2      |  4454.54   |     366.0     |      366.0       | 11227369.54 | 33424031.88 | 22196662.34 |
| 2023.0 |     6.11    |      5.68      |        16.0  